In [1]:
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt

pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)
#接近完美

In [2]:
# desc_data=pd.read_excel("Molecular_Descriptor.xlsx")
#admet_data=pd.read_excel("化合物ADMET.xlsx")
# full_df=pd.merge(desc_data,admet_data,on='SMILES')
# full_df.to_csv("MD_to_ADMET.csv",index=False)

In [3]:
data_df = pd.read_csv("MD_to_ADMET.csv")
data_df.shape

(1974, 735)

In [4]:
target_label = ['Caco-2', 'CYP3A4', 'hERG', 'HOB', 'MN','SMILES']
X = data_df.drop(labels=target_label,axis=1)
y = data_df['MN']

In [6]:
feature_names = X.columns

### 1.分类器准备 XGboost RF随机森林 lightGBM

### 1.1 数据划分

In [7]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

#为了减少误差
# std1 = MinMaxScaler()
# X=std1.fit_transform(X)
# std2 = MinMaxScaler()
# y=std2.fit_transform(y.values.reshape(-1,1))

#训练集 和 验证集 9:1   ;仅仅是分类器
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=666) #这里改名字
#pd.DataFrame(x_train).head()

### 1.2 评价指标

In [8]:
#1.2 评价指标
from sklearn.metrics import confusion_matrix,f1_score,recall_score,accuracy_score,precision_score

def print_score(model,x_train,y_train,x_test,y_test):
    print(model.score(x_train,y_train))
    print(model.score(x_test,y_test))

#误报率的计算
def false_alarm_rate(cm):
    tn, fp, fn, tp = cm.ravel()
    far=fp/(fp+tn)#误报率
    return far

#返回一串字典 输入真实值和预测值
def calc_metrics(true, pred):
    #传进来都是df,查准率precison不关注
    cm=confusion_matrix(true, pred)
    far=false_alarm_rate(cm)*100
    accuracy=accuracy_score(true, pred)*100
    recall=recall_score(true, pred)*100
    f1= f1_score(true, pred)*100
    precise = precision_score(true, pred)*100
    return {'far':far,'precision':precise,'acc':accuracy,'recall':recall,'f1':f1}



In [9]:
# 1.3 写一个function 返回sklearn中 特征重要性排名和分数 的 dataframe
def get_classifer_ranked_feature(clf,feature_names):
    name_im = pd.DataFrame({'importance':clf.feature_importances_,'var':feature_names})
    name_im = name_im.sort_values(by='importance',ascending=False)
    return name_im

In [10]:
%%time
import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV


rfc = RandomForestClassifier()
xgbc =XGBClassifier(max_depth=3,use_label_encoder=False)
#n_estimators=100,max_depth=6, n_jobs=-1,learning_rate=0.15

#针对light-gbm
trn_data = lgb.Dataset(x_train, label = y_train)
val_data = lgb.Dataset(x_test, label = y_test)
lgbm = lgb.LGBMClassifier()

rfc.fit(x_train,y_train)
xgbc.fit(x_train,y_train)
lgbm.fit(x_train,y_train)

[21:18:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Wall time: 5.17 s


LGBMClassifier()

In [11]:
# print_score(xgbc,x_train,y_train,x_test,y_test)
# print_score(rfc,x_train,y_train,x_test,y_test)
# print_score(lgbm,x_train,y_train,x_test,y_test)

In [11]:
import joblib
joblib.dump(lgbm, 'mn_lgbm.model')

['mn_lgbm.model']

In [12]:
trn_pred = lgbm.predict(x_train)
tes_pred =lgbm.predict(x_test)
a=calc_metrics(y_train,trn_pred)
b=calc_metrics(y_test,tes_pred)
print('lgbm')
print(a)
print(b)

lgbm
{'far': 0.0, 'precision': 100.0, 'acc': 100.0, 'recall': 100.0, 'f1': 100.0}
{'far': 18.51851851851852, 'precision': 93.5064935064935, 'acc': 94.94949494949495, 'recall': 100.0, 'f1': 96.64429530201343}


In [13]:
trn_pred = xgbc.predict(x_train)
tes_pred =xgbc.predict(x_test)
a=calc_metrics(y_train,trn_pred)
b=calc_metrics(y_test,tes_pred)
print('xgbc')
print(a)
print(b)

xgbc
{'far': 0.0, 'precision': 100.0, 'acc': 100.0, 'recall': 100.0, 'f1': 100.0}
{'far': 22.22222222222222, 'precision': 92.3076923076923, 'acc': 93.93939393939394, 'recall': 100.0, 'f1': 96.00000000000001}


In [14]:
trn_pred = rfc.predict(x_train)
tes_pred =rfc.predict(x_test)
a=calc_metrics(y_train,trn_pred)
b=calc_metrics(y_test,tes_pred)

In [15]:
print('rfc')
print(a)
print(b)

rfc
{'far': 0.0, 'precision': 100.0, 'acc': 100.0, 'recall': 100.0, 'f1': 100.0}
{'far': 20.37037037037037, 'precision': 92.90322580645162, 'acc': 94.44444444444444, 'recall': 100.0, 'f1': 96.3210702341137}


# 3.选重要的特征

In [16]:
xgbc_fea = get_classifer_ranked_feature(xgbc,feature_names)
rfc_fea = get_classifer_ranked_feature(rfc,feature_names)

In [17]:
xgbc_fea.head(20)

,importance,var
722,0.209254,WTPT-3
660,0.063353,MDEC-24
666,0.055177,MDEO-22
642,0.049277,nHBAcc_Lipinski
601,0.041347,ETA_Epsilon_4
147,0.034251,nssCH2
598,0.029862,ETA_Epsilon_1
48,0.029213,nBondsD
636,0.020350,ETA_EtaP_B_RC
724,0.017845,WTPT-5


In [18]:
rfc_fea.head(20)

,importance,var
724,0.040889,WTPT-5
598,0.027684,ETA_Epsilon_1
12,0.020955,nN
722,0.020709,WTPT-3
642,0.020236,nHBAcc_Lipinski
605,0.020127,ETA_dEpsilon_C
601,0.020062,ETA_Epsilon_4
616,0.019099,ETA_BetaP_s
346,0.015560,minHBa
603,0.015008,ETA_dEpsilon_A
